In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import wandb
import os
from pprint import pprint

In [ ]:

def wandb2pd(exp_runs):
    df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    summary_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    config_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    name_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    summary = [] 
    config = [] 
    name = [] 
    for exp in exp_runs: 
        summary.append(exp.summary._json_dict) 
        config.append({k:v for k,v in exp.config.items() if not k.startswith('_')}) 
        name.append(exp.name)       

    summary_df = pd.DataFrame.from_records(summary) 
    config_df = pd.DataFrame.from_records(config) 
    name_df = pd.DataFrame({'name': name}) 
    df = pd.concat([name_df, config_df, summary_df], axis=1)
    return df

In [ ]:
# colorlist = ["#377eb8", "#ff7f00", "#4daf4a", "darkred", "#984ea3", "y", "k", "w"]
colorlist = ["#ff7f00","#984ea3", "y", "k", "w"]
metrics = ['acc_orig', 'acc_mixed_rand']
metric_names = ['acc_orig', 'acc_mixed_rand']

In [ ]:

optimizer_list = ['momentum_sgd', 'adam']

path_list = [
    f'entity_name/project_momentum_sgd',
    f'entity_name/project_adam',
]



accs = []
ood_accs = []
clr = []

for i in range(len(path_list)):

    p = path_list[i]
    
    spea = []
    kend = []

    api = wandb.Api()
    exp_runs = api.runs(
        path=p,
        filters={'state':'finished'}
        )

    exp_runs
    df = wandb2pd(exp_runs)

    print(p, len(df))

    
    # df['avg_val_acc'] = df['avg_val_acc'].astype(float, errors = 'raise')

    accs = df[metrics[0]].tolist()
    ood_accs = df[metrics[1]].tolist()

    clr = [colorlist[i] for a in range(len(df))]
    plt.scatter(accs, ood_accs, color=clr, s=50, label=optimizer_list[i], alpha=0.5)



#         plt.figure(figsize=(20.0, 15.0))
plt.xlim(0, 1.)
plt.ylim(0, 1.)

plt.xlabel(f'{metric_names[0]}', fontsize=15, labelpad=10)
plt.ylabel(f'{metric_names[1]}', fontsize=15, labelpad=10)

plt.title(f"Background Challenge")

# plt.legend(bbox_to_anchor=(1.05, 0), borderaxespad=0, loc='lower left', fontsize=10).get_frame().set_linewidth(0)
plt.grid(linewidth=1)
plt.savefig(f'figs/scatter/acc_mixed_rand_background.pdf')
plt.show()
